**Install**:
```
pip install -U langchain \
    langgraph==0.1.19 \
    langchain_openai \
    langchainhub \
    langchain_experimental
```

In [1]:
import os
from dotenv import load_dotenv
load_dotenv('./.env')

os.environ["LANGCHAIN_PROJECT"] = "Demo LangGraph 002"

### Model

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

### Tools

In [3]:
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import random

@tool("lower_case", return_direct=True)
def to_lower_case(input:str) -> str:
    """Retorna a entrada em minúsculas"""
    return input.lower()

@tool("random_number", return_direct=True)
def random_number_maker(input:str) -> str:
    """Retorna um número entre 0 e 100, mostrando a palavra 'random'"""
    return random.randint(0, 100)

tools = [to_lower_case, random_number_maker]

In [4]:
from langgraph.prebuilt.tool_executor import ToolExecutor
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function

tool_executor = ToolExecutor(tools)

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

### AgentState

In [5]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

### Nodes

In [6]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    else:
        return "continue"

def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]

    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    print(f"The agent action is {action}")

    response = tool_executor.invoke(action)
    print(f"The tool result is: {response}")

    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {"messages": [function_message]}

### Graph

In [7]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END
    }
)

workflow.add_edge('action', 'agent')

app = workflow.compile()

### Run

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="você é um assistente útil")
user_01 = HumanMessage(content="gere um número aleatório e escreva por extenso em minúsculas.")

inputs = {"messages": [system_message,user_01]}
app.invoke(inputs)

The agent action is tool='random_number' tool_input={'input': 'random'}
The tool result is: 41
The agent action is tool='lower_case' tool_input={'input': 'quarenta e um'}
The tool result is: quarenta e um


{'messages': [SystemMessage(content='você é um assistente útil'),
  HumanMessage(content='gere um número aleatório e escreva por extenso em minúsculas.'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"random"}', 'name': 'random_number'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-4c45b7e7-8774-4290-abbc-340854fb8e20-0'),
  FunctionMessage(content='41', name='random_number'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"quarenta e um"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-a94e3736-c857-4f04-a895-af87b73290ca-0'),
  FunctionMessage(content='quarenta e um', name='lower_case'),
  AIMessage(content='O número aleatório gerado foi 41, e por extenso em minúsculas é "quarenta e um".', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-865b

In [9]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="você é um assistente útil")
user_01 = HumanMessage(content="por favor escreva 'Daniel' em minúsculas")

inputs = {"messages": [system_message,user_01]}
app.invoke(inputs)

The agent action is tool='lower_case' tool_input={'input': 'Daniel'}
The tool result is: daniel


{'messages': [SystemMessage(content='você é um assistente útil'),
  HumanMessage(content="por favor escreva 'Daniel' em minúsculas"),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"input":"Daniel"}', 'name': 'lower_case'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-ea49103c-557f-4802-8596-fc59926569c1-0'),
  FunctionMessage(content='daniel', name='lower_case'),
  AIMessage(content='Aqui está: "daniel"', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-e19f821d-fc59-4854-9454-be479ab98c4f-0')]}

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="você é um assistente útil")
user_01 = HumanMessage(content="o que é machine learning?")

inputs = {"messages": [system_message,user_01]}
app.invoke(inputs)

{'messages': [SystemMessage(content='você é um assistente útil'),
  HumanMessage(content='o que é machine learning?'),
  AIMessage(content='Machine learning é um campo da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem aos computadores aprender e tomar decisões com base em dados, sem serem explicitamente programados. Em vez disso, os algoritmos de machine learning são treinados com conjuntos de dados para identificar padrões e fazer previsões ou tomar decisões com base nesses padrões. Essa capacidade de aprender com os dados e melhorar com a experiência é o que torna o machine learning uma ferramenta poderosa em uma variedade de aplicações, como reconhecimento de padrões, análise de dados, previsão de tendências e muito mais.', response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-f105e0b9-1699-4327-83c1-9a1c2afbb781-0')]}